<a href="https://colab.research.google.com/github/GalalMohammed/SHAI_Training_2022-b_Tasks/blob/main/Task_6_exercises_answered.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![](logo1.jpg)

# **shAI Training 2021 | Level 1**

## Task #6 (End-to-End ML Project {part_2})

## Welcome to the exercises for reviewing second part of end to end ML project.
**Make sure that you read and understand ch2 from the hands-on ML book (page 72 to the end of the chapter ) before start with this notebook.**

**If you stuck with anything reread that part from the book and feel free to ask about anything in the messenger group as you go along.**

 ## Good Luck : )

## first run the following cell for the first part of the project to continue your work 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns
from sklearn.model_selection import train_test_split
from pandas.plotting import scatter_matrix
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

In [ ]:
import os
import tarfile
import urllib
DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    os.makedirs(housing_path, exist_ok=True)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()
    
def load_housing_data(housing_path=HOUSING_PATH):
   csv_path = os.path.join(housing_path, "housing.csv")
   return pd.read_csv(csv_path)
   
fetch_housing_data()
housing = load_housing_data()

rooms_ix, bedrooms_ix, population_ix, household_ix = [
    list(housing.columns).index(col)
    for col in ("total_rooms", "total_bedrooms", "population", "households")]

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room = True):
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self  # nothing else to do
    def transform(self, X, y=None):
        rooms_per_household = X[:, rooms_ix] / X[:, household_ix]
        population_per_household = X[:, population_ix] / X[:, household_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household,
                         bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]
        
train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)
housing = train_set.drop("median_house_value", axis=1)
housing_labels = train_set["median_house_value"].copy()

housing_num = housing.drop("ocean_proximity", axis=1)
num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

num_pipeline = Pipeline([
 ('imputer', SimpleImputer(strategy="median")),
 ('attribs_adder', CombinedAttributesAdder()),
 ('std_scaler', StandardScaler())])

full_pipeline = ColumnTransformer([
 ("num", num_pipeline, num_attribs),
 ("cat", OneHotEncoder(), cat_attribs)])

housing_prepared = full_pipeline.fit_transform(housing)

# 1- Select and Train a Model

# Let’s first train a LinearRegression model 

In [ ]:
# CODE HERE
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(housing_prepared,housing_labels)

LinearRegression()

# First try it out on a few instances from the training set:


In [ ]:
housing_preparedDf = pd.DataFrame(housing_prepared)
some_data = housing_preparedDf.iloc[:5]
some_labels = housing_labels.iloc[:5]

In [ ]:
# CODE HERE
housing_predictions = lin_reg.predict(some_data)
housing_predictions

array([181746.54359616, 290558.74973505, 244957.50017771, 146498.51061398,
       163230.42393939])

# measure this regression model’s RMSE on the whole training set 
* using Scikit-Learn’s mean_squared_error() function:

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
# CODE HERE
lin_mse = mean_squared_error(housing_labels, lin_reg.predict(housing_prepared))
lin_rmse = np.sqrt(lin_mse)
lin_rmse

67593.20745775253

# judge on the RMSE result for this model 
write down your answar 

In [ ]:
lin_rmse / train_set["median_house_value"].mean()

0.32623039827101735

your answer goes here:
The error rate is about 32.62%, which seems bad.

# Let’s train a Decision Tree Regressor model 
## more powerful model

In [ ]:
from sklearn.tree import DecisionTreeRegressor 

In [ ]:
# CODE HERE
tree_reg = DecisionTreeRegressor()
tree_reg.fit(housing_prepared,housing_labels)

DecisionTreeRegressor()

# Now evaluate the model on the training set 
* using Scikit-Learn’s mean_squared_error() function:

In [ ]:
# CODE HERE
housing_predictions = tree_reg.predict(housing_prepared)
tree_mse = mean_squared_error(housing_labels,housing_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

0.0

# Explaine this result 
write down your answar

your answer goes here: The error rate is 0.0%, which means that the model overfits the data.

# Evaluation Using Cross-Validation

1-split the training set into 10 distinct subsets then train and evaluate the Decision Tree model

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
# CODE HERE
scores = cross_val_score(tree_reg, housing_prepared, housing_labels, scoring ="neg_mean_squared_error",cv = 10)
tree_rmse_scores = np.sqrt(-scores)

2- display the resultant scores and calculate its Mean and Standard deviation

In [ ]:
# CODE HERE
print("Scores: ", tree_rmse_scores)
print("Mean: ", tree_rmse_scores.mean())
print("Standard Deviation: ", tree_rmse_scores.std())

Scores:  [63598.89981676 71345.10874655 68467.07801695 70580.31180564
 73006.58913732 66795.76957318 67034.34613325 69738.03923491
 66536.45224391 70063.27361431]
Mean:  68716.5868322773
Standard Deviation:  2630.001575675359


3-repaet the same steps to compute the same scores for the Linear Regression  model 

*notice the difference between the results of the two models*

In [ ]:
# CODE HERE
scores = cross_val_score(lin_reg, housing_prepared, housing_labels, scoring ="neg_mean_squared_error",cv = 10)
lin_rmse_scores = np.sqrt(-scores)
print("Scores: ", lin_rmse_scores)
print("Mean: ", lin_rmse_scores.mean())
print("Standard Deviation: ", lin_rmse_scores.std())

Scores:  [65000.67382615 70960.56056304 67122.63935124 66089.63153865
 68402.54686442 65266.34735288 65218.78174481 68525.46981754
 72739.87555996 68957.34111906]
Mean:  67828.38677377408
Standard Deviation:  2468.091395065227


## Let’s train one last model the RandomForestRegressor.

In [ ]:
# CODE HERE
from sklearn.ensemble import RandomForestRegressor
forest_reg = RandomForestRegressor()
forest_reg.fit(housing_prepared, housing_labels)

RandomForestRegressor()

# repeat the same steps to compute the same scores its Mean and Standard deviation for the Random Forest model

In [ ]:
# CODE HERE
scores = cross_val_score(forest_reg, housing_prepared, housing_labels, scoring ="neg_mean_squared_error",cv = 10)
forest_rmse_scores = np.sqrt(-scores)
print("Scores: ", forest_rmse_scores)
print("Mean: ", forest_rmse_scores.mean())
print("Standard Deviation: ", forest_rmse_scores.std())

Scores:  [47127.20162099 51724.30794266 49995.224645   51728.73208814
 52609.07980417 46932.61629045 47708.88703279 50989.82349064
 49434.74703529 50241.31170665]
Mean:  49849.193165677825
Standard Deviation:  1918.597156843655


# Save every model you experiment with 
*using the joblib library*

In [ ]:
# CODE HERE
import joblib
lin_filename = 'finalized_lin_model.sav'
tree_filename = 'finalized_tree_model.sav'
forest_filename = 'finalized_forest_model.sav'
joblib.dump(lin_reg, lin_filename)
joblib.dump(tree_reg, tree_filename)
joblib.dump(forest_reg, forest_filename)

['finalized_forest_model.sav']

## now you have a shortlist of promising models. You now need to
## fine-tune them!
# Fine-Tune Your Model

## 1- Grid Search
## evaluate all the possible combinations of hyperparameter values for the RandomForestRegressor 
*It may take a long time*

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
# CODE HERE
param_grid = [{'n_estimators': [3,10,30], 'max_features':[2,4,6,8]},{'bootstrap':[False], 'max_features':[2,3,4],'n_estimators':[3,10]}]
grid_search = GridSearchCV(forest_reg,param_grid,cv = 10, scoring = 'neg_mean_squared_error',return_train_score = True)
grid_search.fit(housing_prepared, housing_labels)

GridSearchCV(cv=10, estimator=RandomForestRegressor(),
             param_grid=[{'max_features': [2, 4, 6, 8],
                          'n_estimators': [3, 10, 30]},
                         {'bootstrap': [False], 'max_features': [2, 3, 4],
                          'n_estimators': [3, 10]}],
             return_train_score=True, scoring='neg_mean_squared_error')

with the evaluation scores

In [ ]:
# CODE HERE
print("Best Params: ", grid_search.best_params_)
print("Best Estimator: ", grid_search.best_estimator_)
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"],cvres["params"]):
  print(np.sqrt(-mean_score), params)

Best Params:  {'max_features': 8, 'n_estimators': 30}
Best Estimator:  RandomForestRegressor(max_features=8, n_estimators=30)
63323.62119888364 {'max_features': 2, 'n_estimators': 3}
54204.08370491123 {'max_features': 2, 'n_estimators': 10}
51795.59122645668 {'max_features': 2, 'n_estimators': 30}
59196.05461954129 {'max_features': 4, 'n_estimators': 3}
52380.65644784291 {'max_features': 4, 'n_estimators': 10}
49840.440631412326 {'max_features': 4, 'n_estimators': 30}
58950.72607368776 {'max_features': 6, 'n_estimators': 3}
51660.98762340036 {'max_features': 6, 'n_estimators': 10}
49385.45878546581 {'max_features': 6, 'n_estimators': 30}
57845.09981700682 {'max_features': 8, 'n_estimators': 3}
51584.29179725329 {'max_features': 8, 'n_estimators': 10}
49354.32773053882 {'max_features': 8, 'n_estimators': 30}
61822.51786490256 {'bootstrap': False, 'max_features': 2, 'n_estimators': 3}
53109.32646270088 {'bootstrap': False, 'max_features': 2, 'n_estimators': 10}
59155.588401806366 {'boots

# Analyze the Best Models and Their Errors
1-indicate the relative importance of each attribute

In [ ]:
# CODE HERE
grid_search.best_estimator_.feature_importances_

array([6.78190323e-02, 6.68030759e-02, 4.13943816e-02, 1.50885719e-02,
       1.41124065e-02, 1.40838814e-02, 1.34935929e-02, 3.92085491e-01,
       3.78756927e-02, 1.14555626e-01, 6.92169381e-02, 1.01587976e-02,
       1.37498650e-01, 2.52796133e-04, 2.37021838e-03, 3.19084738e-03])

2-display these importance scores next to their corresponding attribute names:

In [ ]:
# CODE HERE
extra_attribs = ["population_per_household","bedroom_per_room","rooms_per_household"]
num_attribs = list(housing_num.columns)
attributes = num_attribs + extra_attribs + cat_attribs
sorted(zip(grid_search.best_estimator_.feature_importances_,attributes),reverse = True)

[(0.3920854907920388, 'median_income'),
 (0.11455562641035756, 'bedroom_per_room'),
 (0.06921693812148755, 'rooms_per_household'),
 (0.0678190323249126, 'longitude'),
 (0.06680307590280174, 'latitude'),
 (0.041394381603859894, 'housing_median_age'),
 (0.03787569273996867, 'population_per_household'),
 (0.01508857194426263, 'total_rooms'),
 (0.014112406490258012, 'total_bedrooms'),
 (0.01408388137320563, 'population'),
 (0.013493592873146944, 'households'),
 (0.010158797621212416, 'ocean_proximity')]

## Now is the time to evaluate the final model on the test set.
# Evaluate Your System on the Test Set

1-get the predictors and the labels from your test set

In [ ]:
# CODE HERE
Y_test = test_set.drop("median_house_value", axis=1)
Y_test_labels = test_set["median_house_value"].copy()

2-run your full_pipeline to transform the data

In [ ]:
# CODE HERE
Y_test_prepared = full_pipeline.transform(Y_test)

3-evaluate the final model on the test set

In [ ]:
# CODE HERE
final_model = grid_search.best_estimator_
final_predictions = final_model.predict(Y_test_prepared)
final_mse = mean_squared_error(Y_test_labels, final_predictions)
final_rmse = np.sqrt(final_mse)
final_rmse

49546.2552120478

# compute a 95% confidence interval for the generalization error 
*using scipy.stats.t.interval():*

In [ ]:
from scipy import stats

In [ ]:
# CODE HERE
squared_errors = (final_predictions - Y_test_labels) ** 2
stats.t.interval(.95, len(squared_errors) - 1, loc=squared_errors.mean(), scale=stats.sem(squared_errors))

(-213.82347314345293, 2809.2594227558557)

# Great Job!
# #shAI_Club